In [1]:
import math 
import pandas as pd
import random 
import numpy as np
import pickle

In [2]:
# Setting simulation parameters values
N = [30]
#N = [30, 50, 100, 500, 1000]  # population size
D = [0.01, 0.5, 1, 2] # Densities
d = 2 # distance in meters 
T = [1, 2, 5, 7, 10] # infectious time in days
iter = 100 # number of iterations
time = 1440 # exposure time in minutes
#p0 = 0.0034 # enveloppe model hyperparameter 
#lambda_ = 0.3712 # enveloppe model hyperparameter 
#theta_ = math.pi/2 # a constant 

In [3]:
# Define the function to calculate the next values of r 

def f(theta_, r):
    return math.sqrt(r + d**2 - 2 * r * d * math.cos(theta_))

# Remark : I'll use this function later on to optimise the code, not used in this case !!!

In [4]:
# import best model's trace 

with open(r"D:\Manal\Programming\project1\R0_simulation\trace_enveloppe_2m_0.1.pkl", 'rb') as f:
    trace_env = pickle.load(f)


p0_sample = trace_env.posterior["p0"].stack(sample=("chain", "draw")).values
p0_sample = np.random.choice(p0_sample, size=100)


lambda_sample = trace_env.posterior["lambda"].stack(sample=("chain", "draw")).values
lambda_sample = np.random.choice(lambda_sample, size=100)

In [5]:
%%time 

# Perform the simulations

data = []
current_row = None


for k in D: # density
    for n in N: # population size 
        for t in T: # infectious time
            
            for i in range(iter): # iterations
                p0 = p0_sample[i]
                lambda_ = lambda_sample[i]
                results = 0
                
                for m in range(n): # individuals
                    mu = random.uniform(0, 1) # a random number between 0 and 1
                    r0 = math.sqrt((n * mu)/(k * math.pi)) # generate initial radius distance r0
                    r = r0
                    p = 1 - (p0 * math.exp(-lambda_ * r0)) 
                    proba_minute = p
                    for j in range(time):
                        if j == 0:
                            pass
                        else:
                            theta_ = random.uniform(0, 2 * math.pi)
                            r = math.sqrt(r + d**2 - 2 * r * d * math.cos(theta_))  # generate next radius distance per minute 
                            p = 1 - (p0 * math.exp(-lambda_ * r)) # probability per minute
                            proba_minute *= p # product on all probabilities per minute for each ID
                            
                    proba_T = 1 - proba_minute ** t # apply the exponent on infectious time 
                    status = np.random.binomial(1, proba_T) # Bernoulli drawing for infectious status (0 or 1) using the final proba
                    results += status # sum on all Positives 

                # Check if the combination of parameters changes or still the same : to append number of positives as colomns instead of rows

                new_combination = current_row is None or (n, k, t) != (current_row["size"], current_row["density"],  current_row["day"])

                if new_combination:
                    # Create a new row
                    current_row = {"size": n, "density": k,  "day": t}
                    data.append(current_row)

                # Appending the new values of results as an additional columns
                current_row[f"sim{i}"] = str(results)
                    
print("Done")

Done
CPU times: total: 50min 6s
Wall time: 1h 29min


In [7]:
# Convert to dataframe for easy view
df = pd.DataFrame(data)

# Show the header (10 first lines)
df.head(10)

,size,density,day,sim0,sim1,sim2,sim3,sim4,sim5,sim6,...,sim90,sim91,sim92,sim93,sim94,sim95,sim96,sim97,sim98,sim99
0,30,0.01,1,27,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,30,0.01,1,NaN,20,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,30,0.01,1,NaN,NaN,23,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,30,0.01,1,NaN,NaN,NaN,16,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,30,0.01,1,NaN,NaN,NaN,NaN,22,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,30,0.01,1,NaN,NaN,NaN,NaN,NaN,27,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,30,0.01,1,NaN,NaN,NaN,NaN,NaN,NaN,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,30,0.01,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,30,0.01,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,30,0.01,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Save the simulation results into a csv file

df.to_csv("simulation_results_v3.csv", index=False)